<div style="display: flex; align-items: center; gap: 18px; margin-bottom: 15px;">
  <img src="https://files.codebasics.io/v3/images/sticky-logo.svg" alt="Codebasics Logo" style="display: inline-block;" width="130">
  <h1 style="font-size: 34px; color: #1f4e79; margin: 0; display: inline-block;">Codebasics Practice Room - Data Engineering Bootcamp </h1>
</div>


#### 🚀 End-to-End Optimization for Joins & Shuffles

This notebook demonstrates **end-to-end performance optimization**
for Spark pipelines involving **large joins and aggregations**.

The focus is on **reducing shuffle cost**, which is the most expensive
operation in distributed Spark workloads.


## 📂 Dataset

### Fact Table (Large)
- **File:** `sales_orders_large.csv`


**Schema**
- order_id
- order_date
- region
- customer_id
- category
- quantity
- amount

### Dimension Tables (Small)
- `dim_customers_optim.csv`
- `dim_category_group_optim.csv`

> The fact table is much larger than the dimensions.  
> Dimensions are small enough to be **broadcast safely**.


## 🗂️ Scenario

You own a Spark pipeline that:

- Reads **large historical sales data**
- Joins with **multiple dimension tables**
- Performs **aggregations for BI reporting**
- Writes results to a warehouse

As data grows:
- Shuffle stages become very expensive
- Job runtime increases significantly
- Cluster resources are under pressure

Your goal is to **optimize the entire pipeline end-to-end**
to reduce shuffle cost and improve performance.

---

## 🎯 Task

Apply end-to-end optimization strategies to:

1. Reduce data scanned
2. Reduce shuffle volume
3. Optimize joins
4. Improve file layout for downstream BI
5. Make the pipeline scalable and production-ready

---

## 🧩 Assumptions

- Fact table is **much larger** than dimensions
- Dimension tables are **small enough to broadcast**
- Data is queried frequently by **date**
- BI tools read aggregated outputs

> This notebook focuses on **design patterns**, not just code.

---

## 📦 Deliverables

- Optimized join strategy
- Reduced shuffle cost
- Partitioned, analytics-friendly output
- Clear before vs after comparison

### Expected Outcome

| Area | Improvement |
|----|------------|
Shuffle cost | Significantly reduced |
Join performance | Faster via broadcast |
IO | Reduced via partition pruning |
Scalability | Improved |

---

## 🧠 Notes

- Most Spark slowness comes from **shuffles**
- Join strategy matters more than raw compute
- File layout is as important as Spark code
- Optimization is **holistic**, not one-line fixes


## 🧠 Solution Strategy (High-Level)

1. Convert CSV → Parquet/Delta (columnar, compressed)
2. Partition fact data by `order_date`
3. Filter and project early
4. Broadcast small dimensions
5. Tune shuffle partitions
6. Write optimized output for BI


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import broadcast


## 🛢️ Input Data


In [0]:
# Fact table
sales_raw = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("your_data")
)

# Dimension tables
dim_customers = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("your_data")
)

dim_category = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("your_data")
)

display(sales_raw.limit(5))
display(dim_customers.limit(5))
display(dim_category)


## ❌ Naive Version (Slow & Shuffle-Heavy)

Problems:
- CSV scan
- No partition pruning
- No broadcast joins
- Full shuffle on fact table


In [0]:
process_date = "2025-01-15"

fact = (
    sales_raw
        .where(F.col("order_date") == process_date)
        .select("order_date", "region", "customer_id", "category", "amount")
        .alias("f")
)

cust = dim_customers.alias("c")
cat = dim_category.alias("g")


## ✅ Join + clean projection

Convert fact table to **partitioned Parquet**


In [0]:
clean_joined = (
    fact
        .join(broadcast(cust), F.col("f.customer_id") == F.col("c.customer_id"), "left")
        .join(broadcast(cat), F.col("f.category") == F.col("g.category"), "left")
        .select(
            F.col("f.order_date").alias("order_date"),
            F.col("f.region").alias("region"),
            F.col("g.category_group").alias("category_group"),
            F.col("f.amount").alias("amount")
        )
)

## ⚙️ Aggregation


In [0]:
agg = (
    clean_joined
        .groupBy("order_date", "region", "category_group")
        .agg(F.sum("amount").alias("total_amount"))
)

## 💾 Writing Optimized Output for BI


In [0]:
(
    agg.write
       .mode("overwrite")
       .partitionBy("order_date")
       .format("parquet")
       .save("your_directory")
)

## 🔍 Why This Is Faster

- **Partition pruning** → reads only one day
- **Broadcast joins** → avoids shuffling big fact table
- **Column pruning** → reduced IO
- **Shuffle tuning** → fewer, larger shuffle tasks
- **Columnar storage** → faster scans


## ✅ Summary

- Shuffle is the biggest Spark performance bottleneck
- Optimization must be **end-to-end**
- Broadcast joins + partition pruning give the largest gains
- This pattern scales to very large datasets

This is a **production-grade Spark optimization approach**.
